# CS 109A/AC 209A/STAT 121A Data Science: Homework 2
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>
**Due Date: ** Wednesday, September 21st, 2016 at 11:59pm

Download the `IPython` notebook as well as the data file from Vocareum and complete locally.

To submit your assignment, in Vocareum, upload (using the 'Upload' button on your Jupyter Dashboard) your solution to Vocareum as a single notebook with following file name format:

`last_first_CourseNumber_HW2.ipynb`

where `CourseNumber` is the course in which you're enrolled (CS 109a, Stats 121a, AC 209a). Submit your assignment in Vocareum using the 'Submit' button.

**Avoid editing your file in Vocareum after uploading. If you need to make a change in a solution. Delete your old solution file from Vocareum and upload a new solution. Click submit only ONCE after verifying that you have uploaded the correct file. The assignment will CLOSE after you click the submit button.**


Problems on homework assignments are equally weighted. The Challenge Question is required for AC 209A students and optional for all others. Student who complete the Challenge Problem as optional extra credit will receive +0.5% towards your final grade for each correct solution. 

Import libraries

In [2]:
import numpy as np
import pandas as pd
import random
import sklearn as sk
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.cross_validation import train_test_split as sk_split
from sklearn.linear_model import LinearRegression as Lin_Reg
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

---

## Problem 0: Basic Information

Fill in your basic information. 

### Part (a): Your name

[Zambalayev, Timur]

### Part (b): Course Number

[CSCI E-109a]

### Part (c): Who did you work with?

[]

---

## Problem 1: Inside the Models in Scikit-learn

In this problem, we will be implementing K-Nearest Neighbour and simple linear regression for predicting a quantitative variable. We will compare the performance of our implementation with those of Scikit-learn (``sklearn``).

The datasets required for this problem is in the ``dataset`` directory. Each file in the ``dataset`` directory contains a one-dimensional data set, with the first column containing the independent variable X, and the second column containing the dependent variable Y. 

### Part (a): Implement the models by hand
In this part you **may not** use ``sklearn`` for any task.

In the following, you may use ``numpy`` arrays instead of ``pandas`` dataframes.

- Implement a function ``split``, which satifies:
    - input: an ``nx2`` dataframe ``data``, a float ``m``
    - return: an ``nx2`` dataframe ``train`` and an ``nx2`` dataframe ``test``, consisting of ``m`` percent and ``100 - m`` percent of the data, respectively.


- Implement K-Nearest Neighbour for predicting a quantitative variable. That is, write a function, ``knn_predict``, that satisfies:
    - input: an integer ``k``, an ``n x 2`` dataframe training set ``train``, an ``n x 1`` dataframe testing set ``test``
    - return: an ``nx2`` dataframe, whose first column is that of ``test`` and whose second column is the predicted values.



- Implement linear regression for predicting a quantitative variable. That is, write a function ``linear_reg_fit`` that satisfies:
    - input: an ``nx2`` dataframe training set ``train``
    - return: the coefficients of the linear regression model - a float ``slope`` and a float ``intercept``.
    
    
- Write a function ``linear_reg_predict`` that satisfies:
    - input: an ``nx1`` dataframe testing set ``test``, as well as the coefficients of the linear regression model
    - return: an ``nx2`` dataframe, whose first column is that of ``test`` and whose second column is the predicted values.
    
    
- Implement a function ``score`` that satisfies:
    - input: an ``nx2`` dataframe ``predicted``, an ``nx2`` dataframe ``actual`` 
    - return: R^2 coefficient of the fit of the predicted values.

**Solution:**

In [3]:
# split


def split(data, m):
    test_size = 1 - m
    length = len(data)

    indices = list(data.index)
    np.random.shuffle(indices)

    test_length = int(round(test_size * length))
    test_indices = indices[:test_length]
    train_indices = indices[test_length:]

    test = data.loc[test_indices]
    train = data.loc[train_indices]

    return train, test


# knn

def distance(x1, x2):
    return abs(x1 - x2)


def find_best_neighbors_range(sorted_x_list, neighbors_range, test_x):
    current_left_index = neighbors_range[0]
    current_right_index = neighbors_range[1]
    best_range = current_left_index, current_right_index
    while current_right_index < len(sorted_x_list):
        left_distance = distance(test_x, sorted_x_list[current_left_index])
        right_distance = distance(test_x, sorted_x_list[current_right_index])
        if left_distance <= right_distance:
            break
        best_range = current_left_index, current_right_index
        current_left_index += 1
        current_right_index += 1
    if best_range[0] == neighbors_range[0]:
        # we didn't find better neighbors on the right, so we'll search left
        current_left_index = neighbors_range[0] - 1
        current_right_index = neighbors_range[1] - 1
        while current_left_index >= 0:
            left_distance = distance(test_x, sorted_x_list[current_left_index])
            right_distance = distance(test_x, sorted_x_list[current_right_index])
            if left_distance >= right_distance:
                break
            best_range = current_left_index, current_right_index
            current_left_index -= 1
            current_right_index -= 1
    return best_range


def get_initial_range(k, insertion_index, length):
    left_index = max(0, insertion_index - k / 2)
    right_index = left_index + k
    if right_index > length:
        right_index = length
        left_index = right_index - k
    return left_index, right_index


def find_nearest_neighbors(k, sorted_x_list, test_x):
    insertion_index = bisect.bisect_left(sorted_x_list, test_x)
    initial_range = get_initial_range(k, insertion_index, len(sorted_x_list))
    return find_best_neighbors_range(sorted_x_list, initial_range, test_x)


def knn_predict_one_point(k, sorted_x_list, sorted_y_list, test_x):
    neighbors_range = find_nearest_neighbors(k, sorted_x_list, test_x)
    total = 0.0
    for index in range(neighbors_range[0], neighbors_range[1]):
        total += sorted_y_list[index]
    return total / k


def knn_predict(k, train, test):
    sorted_train = train.sort_values(by='x')
    sorted_x_list = sorted_train['x'].tolist()
    sorted_y_list = sorted_train['y'].tolist()
    predicted_test = test.copy()

    predicted_test['y'] = [knn_predict_one_point(k, sorted_x_list, sorted_y_list, row['x'])
                           for index, row in test.iterrows()]
    return predicted_test


# linear regression

def linear_reg_fit(train):
    x_list = train['x'].tolist()
    y_list = train['y'].tolist()

    x_mean = np.mean(x_list)
    y_mean = np.mean(y_list)

    numerator_sum = 0.0
    denominator_sum = 0.0
    for x, y in izip(x_list, y_list):
        x_diff = x - x_mean
        numerator_sum += x_diff * (y - y_mean)
        denominator_sum += x_diff ** 2

    slope = numerator_sum / denominator_sum
    intercept = y_mean - slope * x_mean
    return slope, intercept


def linear_reg_predict(test, slope, intercept):
    predicted_test = test.copy()
    predicted_test['y'] = [intercept + slope * row['x'] for index, row in test.iterrows()]
    return predicted_test


# score

def score(predicted, actual):
    rss = 0.0
    tss = 0.0
    actual_y_list = actual['y'].tolist()
    actual_y_mean = np.mean(actual_y_list)
    for predicted_value, actual_value in izip(predicted['y'].tolist(), actual_y_list):
        rss += (actual_value - predicted_value) ** 2
        tss += (actual_value - actual_y_mean) ** 2
    return 1.0 - rss / tss


### Part (b): Compare with ``sklearn``

- Load the contents of ``dataset_1_full.txt`` into a ``pandas`` dataframe, or ``numpy`` array. 


- Use your functions from Part (a) to split the data into training and testing sets (70-30). Evaluate how KNN and linear regression each perform on this dataset.


- Use ``sklearn`` to split the data into training and testing sets (70-30). Use ``sklearn`` to evaluate how KNN and linear regression each perform on this dataset.


- Use Python's ``time`` library to measure how well your implementations compare with that of ``sklearn``. What can you do (algorithmically or codewise) to make your implementation faster or more efficient?

**Solution:**

In [8]:
import bisect
from itertools import izip


def generate_k_list():
    k_list = list(range(1, 6))
    k_list.extend(range(10, 21, 5))
    k_list.extend(range(30, 51, 10))
    k_list.extend(range(75, 101, 25))
    k_list.extend(range(150, 351, 50))
    return k_list


def evaluate_our_implementation(df, k_list):
    print 'our implementation'
    train, test = split(df, 0.7)

    test_x = test[['x']]
    for k in k_list:
        start = time.time()
        predicted_test = knn_predict(k, train, test_x)
        s = score(predicted_test, test)
        elapsed_time = time.time() - start
        print 'KNN, k: %d, score: %.3f, time: %.2f' % (k, s, elapsed_time)

    start = time.time()
    slope, intercept = linear_reg_fit(train)
    predicted_test = linear_reg_predict(test_x, slope, intercept)
    s = score(predicted_test, test)
    elapsed_time = time.time() - start
    print 'linear regression, score: %.3f, time: %.2f\n' % (s, elapsed_time)


def reshape(df, column_name):
    return df[column_name].reshape((len(df), 1))


def evaluate_sklearn_implementation(df, k_list):
    print 'sklearn implementation'
    train, test = sk_split(df, train_size=0.7)

    x_train = reshape(train, 'x')
    y_train = reshape(train, 'y')
    x_test = reshape(test, 'x')
    y_test = reshape(test, 'y')
    for k in k_list:
        start = time.time()
        neighbors = KNN(n_neighbors=k)
        neighbors.fit(x_train, y_train)
        s = neighbors.score(x_test, y_test)
        elapsed_time = time.time() - start
        print 'KNN, k: %d, score: %.3f, time: %.2f' % (k, s, elapsed_time)

    start = time.time()
    regression = Lin_Reg()
    regression.fit(x_train, y_train)
    s = regression.score(x_test, y_test)
    elapsed_time = time.time() - start
    print 'linear regression, score: %.3f, time: %.2f' % (s, elapsed_time)


def compare_with_sklearn():
    np.random.seed(1090)

    df = pd.read_csv('dataset/dataset_1_full.txt')
    k_list = generate_k_list()

    evaluate_our_implementation(df, k_list)
    evaluate_sklearn_implementation(df, k_list)

    
compare_with_sklearn()

our implementation
KNN, k: 1, score: -0.396, time: 0.01
KNN, k: 2, score: -0.027, time: 0.01
KNN, k: 3, score: 0.205, time: 0.01
KNN, k: 4, score: 0.244, time: 0.00
KNN, k: 5, score: 0.266, time: 0.01
KNN, k: 10, score: 0.338, time: 0.00
KNN, k: 15, score: 0.342, time: 0.01
KNN, k: 20, score: 0.382, time: 0.01
KNN, k: 30, score: 0.417, time: 0.01
KNN, k: 40, score: 0.402, time: 0.01
KNN, k: 50, score: 0.421, time: 0.00
KNN, k: 75, score: 0.417, time: 0.01
KNN, k: 100, score: 0.413, time: 0.01
KNN, k: 150, score: 0.397, time: 0.01
KNN, k: 200, score: 0.364, time: 0.01
KNN, k: 250, score: 0.284, time: 0.01
KNN, k: 300, score: 0.181, time: 0.01
KNN, k: 350, score: -0.002, time: 0.01
linear regression, score: 0.418, time: 0.00

sklearn implementation
KNN, k: 1, score: -0.078, time: 0.00
KNN, k: 2, score: 0.177, time: 0.00
KNN, k: 3, score: 0.231, time: 0.00
KNN, k: 4, score: 0.259, time: 0.00
KNN, k: 5, score: 0.302, time: 0.00
KNN, k: 10, score: 0.327, time: 0.00
KNN, k: 15, score: 0.367,

***What can you do to make your implementation faster?***

My current implmentation is as fast is sklearn's at least on this dataset. 
The optimizations I made in comparison to a possible naive implementation were:
* I sorted the train dataframe by 'x' values so that it's easier/faster to locate k nearest neighbors. I used binary search to find the insertion point in the sorted array. Then I would start with a range of length k centered around that insertion point. Then I would try to improve that range by trying to move to the right and to the left.
* Another optimization I made is I passed individual sorted 'x' and 'y' value lists for the train set so that not to do dataframe indexing inside a very nested loop. It improved the performance dramatically.

## Problem 2: Handling Missing Data

In this problem, we will be handling the problem of datasets with missing values. Clearly, we cannot simply remove entire rows or columns that contain missing values. In this problem, we explore two different ways to fill in missing values.

The datasets required for this problem is in the ``dataset`` directory. Each file in the ``dataset`` directory contains a one-dimensional data set, with the first column containing the independent variable X, and the second column containing the dependent variable Y.

The files ``dataset_1_missing.txt`` to ``dataset_6_missing.txt`` contains rows that are missing their y-values, where as ``dataset_1_full.txt`` to ``dataset_6_full.txt`` contain datasets with all y-values correctly filled in.

In this problem, you **may not** use ``sklearn`` or build-in ``pandas`` functions to **directly fill in missing values**. Usage of these libraries/packages for related tasks is fine.

### Part (a): Model Based Data Imputation

- Describe in detail how predictive models for data (like KNN and simple linear regression) can be used to fill in missing values in a data set.


- Implement your scheme. That is, write code (preferably a function ``fill`` or two functions ``fill_knn``, ``fill_lin_reg``), which takes an ``n x 2`` dataframe or array with values missing in the 2nd column and fills in these values using KNN and linear regression. 


- You need to, also, write code to evaluate the quality of the values you've filled in.

**Solution:**

### Part (b): Which Model is Better?

- For datasets ``dataset_1_missing.txt`` to ``dataset_6_missing.txt``, compare the result of filling in the missing values using KNN and linear regression, using both the R^2 coefficient as well as data visualization (the correct y-values are contained in ``dataset_1_full.txt`` to ``dataset_6_full.txt``).. 


- Use your analysis to form conjectures regarding the conditions under which KNN performs better than linear regression, under which linear regression performs better than KNN and under which both perform equally (well or poorly). Explain in detail exactly what might cause each model to fail or perform well. 



- Using ``dataset_1_missing.txt``, explain the impact of the choice of $k$ on the performance of KNN. 


Use numerical analysis and data visualization to support every part of your argument.

**Solution:**

---

## Problem 3: Is the Best (Linear Model) Good Enough?

In this problem, we will specifically look at conditions under which linear regression excels or fails.

The datasets required for this problem is in the ``dataset`` directory. Each file in the ``dataset`` directory contains a one-dimensional data set, with the first column containing the independent variable X, and the second column containing the dependent variable Y.

### Part (a): Introduction to Residual Plots

- Read ``dataset_1_full.txt``. Visualize the dataset and make some initial observations.


- For this data set, what can you say about the following linear fits: 

    1. slope = 0.4, intercept = 0.2
    2. slope = 0.4, intercept = 4
    3. linear regression model


- In each case, visualize the fit, compute the residuals, and make a residual plot of predicted values along with  residuals, as well as a residual histogram. What do these plots reveal?  


- Calculate the R^2 coefficient for all three fits. What do the erors reveal? How do they compare to the residual plots?

**Solution:**

### Part (b): What do Residual Plots Reveal?

- Read datasets ``dataset_2_full.txt`` through ``dataset_6_full.txt``. In each case, visualize the fit, compute the residuals, and make a residual plot of predicted values along with  residuals, as well as a residual histogram. What do these plots reveal about the fit of the model? 


- Calculate the R^2 coefficient each fit. What do the erors reveal? How do they compare to the residual plots?


- Based on your analysis, form conjectures regarding the precise relationship between the residual plots and the fit of the linear regression model. Conjecture on the precise conditions under which linear regression model is an appropriate model for a given dataset.

**Solution:**

---

## Challenge Problem: Combining Random Variables

This problem, we explore the distirbution of random variables that result from combining other random variables.

### Part (a): Adding Two Uniformly Distributed Variables

Consider the independent random variables $X\sim U(0, 1)$ and $Y\sim U(0, 1)$. Let $Z$ be the random variable $Z = X + Y$. 

What is the distribution of $Z$ (give the pdf for Z)? You should fully explain and support your conlusion. 

**Hint:** your solution can be a combination of experimentation, empirical evidence and/or algebra

**Solution:**

### Part (b): Adding Multiple Uniformly Distributed Variables

Consider three independent random variables $X_1, X_2, X_3 \sim U(0, 1)$. Let $Z$ be the random variable $Z = X_1 + X_2 + X_3$. 

What is the distribution of $Z$? What if you add 10 or 12 independent (standard) uniformly distributed variables? Conjecture on the distribution of 
$$
Z = \lim_{n\to \infty} \sum_{i=1}^n X_i
$$
where $\left\{X_i \right\}$ are independent (standard) uniformly distributed variables.

**Hint:** your solution can be a combination of experimentation, empirical evidence and/or algebra

**Solution:**

### Part (c): Combining Normally Distributed Variables

Consider the independent random variables $X\sim \mathcal{N}(0, 1)$ and $Y\sim \mathcal{N}(0, 1)$. Let $Z$ be the random variable $Z = X + Y$. 

What is the distribution of $Z$ (give the pdf for Z)? You should fully explain and support your conlusion. 

**Hint:** use properties of expected value and some experimentation.

**Solution:**

### Part (d): Product of Normally Distributed Variables

Is the product of two normally distributed variables a normally distributed variable? You should fully explain and support your conlusion. 

**Solution:**

---